In [1]:

import pandas as pd
import numpy as np 
import nltk
import string
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc,accuracy_score,f1_score,precision_score,recall_score
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import precision_score,recall_score,auc

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
train_data = pd.read_csv("/Users/sarath/Documents/Final Project Data/train_values.csv")
train_labels = pd.read_csv("/Users/sarath/Documents/Final Project Data/train_labels.csv")
test_labels = pd.read_csv("/Users/sarath/Documents/Final Project Data/test_values.csv")

In [6]:
building_damage1 = train_data.merge(train_labels, how = 'inner', on = 'building_id')

In [7]:
from functools import reduce

In [8]:
building_damage1 = building_damage1.drop(columns ="has_secondary_use")

In [9]:
building_damage1 = building_damage1.drop(columns ="has_secondary_use_agriculture")

In [10]:
building_damage = building_damage1[building_damage1['age'] <= 250]

In [11]:
# utility function to preprocess data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer,LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
def preprocess_num_data(X_train,num_data,X_test):
    std=Normalizer()
    std.fit(train_data)
    transformed_input=std.transform(building_damage)
    transformed_test=std.transform(test_labels)
    return transformed_input,transformed_test
def preprocess_cat_data(building_damage,test_labels):
    std=LabelEncoder()
    for col in train_data.columns:
        col_list_train=list(map(str,building_damage[col].values))
        #print(col_list_train)
        col_list_test=list(map(str,test_lables[col].values))
        std.fit(col_list_train)
        col_list_train=std.transform(col_list_train)
        col_list_test=std.transform(col_list_test)
        building_damage[col]=col_list_train
        test_data[col]=col_list_test
    return building_damage,test_data

In [12]:
num_data=['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq','area_percentage', 'age','height_percentage','count_families']
cat_data=['land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position','legal_ownership_status',
       'plan_configuration'] 
bin_data=['has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
         'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']

In [13]:
# filtering numerical data
num_train=building_damage[num_data]
num_test=test_labels[num_data]
# filtering categorical data
cat_train=building_damage[cat_data]
cat_test=building_damage[cat_data]
# filtering binary data
bin_train=building_damage[bin_data]
bin_test=test_labels[bin_data]

In [14]:
cat_feats = ['land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position','legal_ownership_status',
       'plan_configuration']

In [15]:
building_damage['damage_grade'] = building_damage['damage_grade'].astype('category')

In [16]:
train_final = pd.get_dummies(building_damage,columns=cat_feats,drop_first=True)
test_final = pd.get_dummies(test_labels,columns=cat_feats,drop_first=True)

In [17]:
train_final['damage_grade'].dtype

CategoricalDtype(categories=[1, 2, 3], ordered=False)

### Train Test Split

In [18]:
from sklearn.model_selection import train_test_split

In [23]:
X = train_final.drop('damage_grade',axis=1)
y = train_final['damage_grade']

train_final.shape

(259211, 60)

In [21]:
X = train_final.drop('damage_grade',axis=1)
y = train_final['damage_grade']


(259211, 60)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)

In [25]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [33]:
y_train  = list(y_train)

In [29]:
from sklearn.svm import SVC

In [30]:
svc_model = SVC()

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

In [26]:
from sklearn.model_selection import GridSearchCV

In [27]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001]} 

In [ ]:
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train,y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] C=0.1, gamma=1 ..................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................................. C=0.1, gamma=1, total=174.5min
[CV] C=0.1, gamma=1 ..................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 191.9min remaining:    0.0s


[CV] ................................... C=0.1, gamma=1, total=98.7min
[CV] C=0.1, gamma=1 ..................................................
